<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/NLP_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [341]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

Mon Dec 12 14:31:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    27W /  70W |  15102MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [342]:
!pip3 install transformers
!pip3 install datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [343]:
# basics 
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# transformers 
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from transformers import CamembertModel, CamembertTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# plot 
import matplotlib.pyplot as plt 
import seaborn as sns 

# torch 
import torch
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# nltk 
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [344]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Lecture et analyse pandas

In [345]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 10
VALID_BATCH_SIZE = 10
EPOCHS = 1
LEARNING_RATE = 1e-05

LEN_TEXT = 300
OVERLAP = 50

In [346]:
train_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_train.csv'
test_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_test.csv'

df = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# suppression des colonnes inutiles 
df = df.drop(['Unnamed: 0', 'target_name'], axis = 1)
df_test = df_test.drop(['Unnamed: 0', 'target_name'], axis = 1)

In [347]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('french'))

def clean_text(text):
    text = text.lower() # lowercase text
    #text = REPLACE_BY_SPACE_RE.sub(' ', text)
    #text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [348]:
df['data'] = df['data'].apply(clean_text)
df_test['data'] = df_test['data'].apply(clean_text)

In [349]:
df.head()

,data,label
0,22e jour consécutif grève reconductible contre...,0
1,"depuis plusieurs mois, initiatives chercheurs ...",0
2,google vient d'introduire mise jour applicatio...,0
3,portrait. chacun s’empresse autour d’elle tand...,0
4,"« n’y a risque pénurie » carburant, a déclaré ...",0


In [350]:
def get_split(text1):
    l_total = []
    l_parcial = []
    if len(text1.split())//(LEN_TEXT - OVERLAP) >0:
        n = len(text1.split())//(LEN_TEXT - OVERLAP)
    else: 
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:LEN_TEXT]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*(LEN_TEXT - OVERLAP):w*(LEN_TEXT - OVERLAP) + LEN_TEXT]
            l_total.append(" ".join(l_parcial))
    return l_total

In [351]:
df['text_split'] = df['data'].apply(get_split)
df['len_split'] = df['text_split'].apply(lambda x: len(x))

df_test['text_split'] = df_test['data'].apply(get_split)
df_test['len_split'] = df_test['text_split'].apply(lambda x: len(x))

In [352]:
df['len_split'].value_counts()

1     1245
2      129
3       45
4       24
7        5
6        5
5        2
12       1
8        1
14       1
Name: len_split, dtype: int64

In [353]:
df_test['len_split'].value_counts()

1     419
2      42
3      11
5       6
4       5
7       1
6       1
11      1
Name: len_split, dtype: int64

In [354]:
# verification du fonctionnement

for index, row in df.iterrows():
  if len(row['text_split']) > 1: 
    print(index)
    break

print(df['text_split'][31][0].split()[(LEN_TEXT - OVERLAP):])
print(df['text_split'][31][1].split()[:OVERLAP])

12
['natalia', 'rybczynski', 'paléobiologiste.', 'suivez', 'recherche', 'fossiles', 'étudiants', "l'université", 'carleton', 'lors', "l'expédition", 'students', 'ice', 'antarctic', '2011.', "l'équipe", 'visite', 'îles', 'déception', 'seymour.', '©', 'musée', 'canadien', 'nature', 'mémoire', 'isotopes', 'calcium', 'menés', 'benjamin', 'linzmeier', 'andrew', 'd.', 'jacobson', "l'université", 'northwestern', 'chicago,', 'chercheurs', 'basé', 'leurs', 'travaux', 'collecte', 'fossiles', 'retrouvés', 'formation', 'géologique', 'célèbre', 'trouve', "l'île", 'seymour,', 'formation']
['natalia', 'rybczynski', 'paléobiologiste.', 'suivez', 'recherche', 'fossiles', 'étudiants', "l'université", 'carleton', 'lors', "l'expédition", 'students', 'ice', 'antarctic', '2011.', "l'équipe", 'visite', 'îles', 'déception', 'seymour.', '©', 'musée', 'canadien', 'nature', 'mémoire', 'isotopes', 'calcium', 'menés', 'benjamin', 'linzmeier', 'andrew', 'd.', 'jacobson', "l'université", 'northwestern', 'chicago,', 

In [355]:
def create_df(df): 
  train_l = []
  label_l = []
  for idx,row in df.iterrows():
      for l in row['text_split']:
          train_l.append(l)
          label_l.append([1 if row['label'] == i else 0 for i in range(2)])

  return pd.DataFrame({'data':train_l, 'label':label_l})

In [356]:
cleaned_df = create_df(df)
cleaned_df_test = create_df(df_test)

In [357]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, is_target = True):
        self.tokenizer = tokenizer
        self.df = dataframe
        self.text = dataframe.data
        self.max_len = max_len
        if is_target: 
          self.targets = self.df.label
        else: 
          self.targets = None

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        if self.targets is None: 
          return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(mask, dtype=torch.long),
              'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
          }
        else: 
          return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(mask, dtype=torch.long),
              'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
              'targets': torch.tensor(self.targets[index], dtype=torch.float)
          }

In [358]:
tranfo_name = "cmarkea/distilcamembert-base"
tokenizer = AutoTokenizer.from_pretrained(tranfo_name)

In [359]:
train_size = 0.8
train_dataset=cleaned_df.sample(frac=train_size,random_state=200)
test_dataset=cleaned_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(cleaned_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(cleaned_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(cleaned_df_test, tokenizer, MAX_LEN)

FULL Dataset: (1843, 2)
TRAIN Dataset: (1474, 2)
TEST Dataset: (369, 2)


In [360]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [361]:
class BERTClass(torch.nn.Module):
    def __init__(self):
      super(BERTClass, self).__init__()
      self.l1 = CamembertModel.from_pretrained(tranfo_name)
      self.l3 = torch.nn.Linear(768, 2) #2 = binary classification
    
    def forward(self, ids, mask, token_type_ids):
      output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
      output = self.l3(output_1['pooler_output'])

      return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream

BERTClass(
  (l1): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0): CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [362]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [363]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        token_type_ids = data['token_type_ids'].to(device)
        targets = data['targets'].to(device)
        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%10==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [364]:
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  0.7094480991363525
Epoch: 0, Loss:  0.46974673867225647
Epoch: 0, Loss:  0.5431113243103027
Epoch: 0, Loss:  0.4170994758605957
Epoch: 0, Loss:  0.3294007480144501
Epoch: 0, Loss:  0.20950499176979065
Epoch: 0, Loss:  0.11740458011627197
Epoch: 0, Loss:  0.19770044088363647
Epoch: 0, Loss:  0.28697702288627625
Epoch: 0, Loss:  0.11256702989339828
Epoch: 0, Loss:  0.2144276648759842
Epoch: 0, Loss:  0.06517084687948227
Epoch: 0, Loss:  0.19719281792640686
Epoch: 0, Loss:  0.11961408704519272
Epoch: 0, Loss:  0.05906033515930176
Epoch: 0, Loss:  0.05943263694643974
Epoch: 0, Loss:  0.19426369667053223
Epoch: 0, Loss:  0.10858414322137833
Epoch: 0, Loss:  0.32572126388549805


In [365]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
      for _, data in enumerate(testing_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)
        fin_targets.extend(targets.cpu().detach().numpy().tolist())

        m = torch.nn.Softmax(dim=1)
        fin_outputs.extend(torch.round(m(outputs)).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [366]:
from sklearn import metrics
outputs, targets = validation()
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.9590163934426229
F1 Score (Micro) = 0.9590163934426229
F1 Score (Macro) = 0.9554316409046939


In [367]:
cleaned_df_test['pred'] = outputs
cleaned_df_test.head()

,data,label,pred
0,président groupe lrem a pris toutes pincettes....,"[1, 0]","[1.0, 0.0]"
1,"villes françaises, qualité l'air meilleure moi...","[1, 0]","[1.0, 0.0]"
2,"cop25, vient s'achever, laisse goût amer certa...","[1, 0]","[1.0, 0.0]"
3,"catherine abreu, climate action network . cop2...","[1, 0]","[1.0, 0.0]"
4,donné signe clair d'une telle intention 2020. ...,"[1, 0]","[1.0, 0.0]"


In [368]:
cleaned_df_test.loc[0]['pred']

[1.0, 0.0]

In [369]:
pos = 0

df_test['pred'] = [list() for x in range(len(df_test.index))]

for idx,row in df_test.iterrows():
  for i in range(row['len_split']): 
    row['pred'].append(cleaned_df_test.loc[pos]['pred'])
    pos += 1

In [370]:
for idx, row in df_test.iterrows(): 
  if row['len_split'] > 1: 
    print(row['pred'], row['label'])

[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0]] 0
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0

In [371]:
checkpoint = {'model': BERTClass(),
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream

In [372]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    return model

In [373]:
the_model = load_checkpoint('checkpoint.pth')

In [375]:
news = "hello there"

d = {'data': [news]}
df = pd.DataFrame(data=d)

test_set = CustomDataset(df, tokenizer, MAX_LEN, is_target = False)

In [ ]:
_params = {'batch_size': 1,
                'shuffle': True,
                'num_workers': 0
                }

_loader = DataLoader(test_set, **_params)

In [ ]:
the_model.eval()
the_model.to(device)
for _,data in enumerate(_loader, 0):
  ids = data['ids'].to(device)
  mask = data['mask'].to(device)
  token_type_ids = data['token_type_ids'].to(device)
  
  outputs = the_model(ids, mask, token_type_ids)
  _, predicted = torch.max(outputs.data, 1)

  print(predicted)